<a href="https://colab.research.google.com/github/nirmalpate/E_COMMERCE_A-B-TEST/blob/main/E_COMMERCE_A_B_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import numpy as np
from scipy import stats

In [50]:
np.random.seed(3000)
total_users = 500  # arbitrary large number
df = pd.DataFrame({
    'user_id': range(1, total_users + 1),
    'group': np.random.choice(['Control','Treatment'], total_users),
    'base_price': np.random.uniform(20, 150, total_users),
    'items_bought': np.random.choice([1,2,3,4], total_users, p=[0.5,0.3,0.15,0.05])
})

In [56]:
df['revenue_per_item'] = df['base_price']
df['revenue_per_item'] = df['base_price']
df['total_revenue'] = df['revenue_per_item'] * df['items_bought'] * np.where(df['group']=='Treatment', 1.3, 1.0)

# Conversion / guardrail metric
df['converted'] = np.where(df['group']=='Control',
                           np.random.choice([0,1], total_users, p=[0.5,0.5]),
                           np.random.choice([0,1], total_users, p=[0.45,0.55]))

df

alpha = 0.05
power = 0.9
mde = 0.30

base_avg = df['total_revenue'].mean()
variance = df['total_revenue'].var()
std_dev = np.sqrt(variance)
delta = mde * base_avg

z_alpha = stats.norm.ppf(1 - alpha/2)
z_beta  = stats.norm.ppf(power)

n_req = ((z_alpha + z_beta)**2 * 2 * variance) / (delta**2)
n_req = int(np.ceil(n_req))
print(f"Required sample size per group (Revenue): {n_req}")

Required sample size per group (Revenue): 130


In [57]:
import pandas as pd
import numpy as np
from scipy import stats


# -----------------------------
# 1. PARAMETERS AND DATA GENERATION
# -----------------------------
np.random.seed(3000)

# Generate single dataset for both sample size calculation and experiment
total_users = 500  # arbitrary large number
df = pd.DataFrame({
    'user_id': range(1, total_users + 1),
    'group': np.random.choice(['Control','Treatment'], total_users),
    'base_price': np.random.uniform(20, 150, total_users),
    'items_bought': np.random.choice([1,2,3,4], total_users, p=[0.5,0.3,0.15,0.05])
})

# Revenue calculation (Treatment spends 30% more)
df['revenue_per_item'] = df['base_price']
df['total_revenue'] = df['revenue_per_item'] * df['items_bought'] * np.where(df['group']=='Treatment', 1.3, 1.0)

# Conversion / guardrail metric
df['converted'] = np.where(df['group']=='Control',
                           np.random.choice([0,1], total_users, p=[0.5,0.5]),
                           np.random.choice([0,1], total_users, p=[0.45,0.55]))

df


# -----------------------------
# 2. SAMPLE SIZE CALCULATION (USING GENERATED DATA)
# -----------------------------
alpha = 0.05
power = 0.9
mde = 0.30  # 30% increase in revenue

# Compute mean and variance from single dataset
base_rev_avg = df['total_revenue'].mean()
variance = df['total_revenue'].var()
delta = base_rev_avg * mde

z_alpha = stats.norm.ppf(1 - alpha/2)
z_beta  = stats.norm.ppf(power)

n_req = ((z_alpha + z_beta)**2 * 2 * variance) / (delta**2)
n_req = int(np.ceil(n_req))
print(f"Required sample size per group (Revenue): {n_req}")


# -----------------------------
# 3. SUCCESS METRIC: Revenue per user (T-Test)
# -----------------------------
rev_c = df[df['group']=='Control']['total_revenue']
rev_t = df[df['group']=='Treatment']['total_revenue']

t_stat, p_val_rev = stats.ttest_ind(rev_c, rev_t)
print('T-Statistic:', t_stat)
print('p-value:', p_val_rev)

if p_val_rev < 0.05:
    print('Reject the null hypothesis. There is a significant increase in revenue per user.')
else:
    print('Fail to reject the null hypothesis. There is no significant increase in revenue per user.')


# -----------------------------
# 4. GUARDRAIL METRIC: Conversion Rate (Chi-Square Test)
# -----------------------------
contingency = pd.crosstab(df['group'], df['converted'])
chi2, p_val_conv, dof, ex = stats.chi2_contingency(contingency)
print('Chi-Square Statistic:', chi2)
print('p-value:', p_val_conv)

if p_val_conv < 0.05:
    print('Reject the null hypothesis. There is a significant difference in conversion rate between groups.')
else:
    print('Fail to reject the null hypothesis. There is no significant difference in conversion rate between groups.')


# -----------------------------
# 5. MDE AND DELTA SUMMARY
# -----------------------------
print(f'Baseline Revenue per user: €{base_rev_avg:.2f}')
print(f'MDE (30% of baseline): €{delta:.2f}')


Required sample size per group (Revenue): 130
T-Statistic: -4.664554593524094
p-value: 3.979147470397966e-06
Reject the null hypothesis. There is a significant increase in revenue per user.
Chi-Square Statistic: 0.0
p-value: 1.0
Fail to reject the null hypothesis. There is no significant difference in conversion rate between groups.
Baseline Revenue per user: €166.76
MDE (30% of baseline): €50.03


In [ ]:
# post OBSERVED A/B TEST

In [58]:
import pandas as pd
import numpy as np
from scipy import stats

# -----------------------------
# 1. DATA GENERATION (POST-OBSERVED A/B TEST)
# -----------------------------
np.random.seed(3000)

# Total users
total_users = 100  # you can increase for bigger dataset
groups = ['Control'] * (total_users // 2) + ['Treatment'] * (total_users // 2)

df = pd.DataFrame({
    'user_id': range(1, total_users + 1),
    'group': groups,
    'base_price': np.random.uniform(20, 150, total_users),
    'items_bought': np.random.choice([1,2,3,4], total_users, p=[0.5,0.3,0.15,0.05])
})

# Revenue calculation: Treatment users spend 30% more
df['total_revenue'] = df['base_price'] * df['items_bought'] * np.where(df['group']=='Treatment', 1.3, 1.0)

# Conversion (guardrail metric)
df['converted'] = np.where(df['group']=='Control',
                           np.random.choice([0,1], total_users, p=[0.5,0.5]),
                           np.random.choice([0,1], total_users, p=[0.45,0.55]))

# Post-observed revenue after conversion
df['post_revenue'] = df['total_revenue'] * df['converted']

# -----------------------------
# 2. SAMPLE SIZE (from observed data)
# -----------------------------
alpha = 0.05
power = 0.9
mde = 0.30  # 30% revenue increase
base_rev_avg = df['post_revenue'].mean()
variance = df['post_revenue'].var()
delta = base_rev_avg * mde

z_alpha = stats.norm.ppf(1 - alpha/2)
z_beta = stats.norm.ppf(power)

n_req = ((z_alpha + z_beta)**2 * 2 * variance) / (delta**2)
n_req = int(np.ceil(n_req))

print(f"Required sample size per group: {n_req}")

# -----------------------------
# 3. SUCCESS METRIC: Revenue per user (T-Test)
# -----------------------------
rev_c = df[df['group']=='Control']['post_revenue']
rev_t = df[df['group']=='Treatment']['post_revenue']

t_stat, p_val_rev = stats.ttest_ind(rev_c, rev_t)
print('\nRevenue per User Test (Success Metric)')
print(f'T-Statistic: {t_stat:.3f}, p-value: {p_val_rev:.4f}')

if p_val_rev < 0.05:
    print('Reject the null hypothesis: significant increase in revenue per user.')
else:
    print('Fail to reject null hypothesis: no significant increase in revenue per user.')

# -----------------------------
# 4. GUARDRAIL METRIC: Conversion Rate (Chi-Square Test)
# -----------------------------
contingency = pd.crosstab(df['group'], df['converted'])
chi2, p_val_conv, dof, ex = stats.chi2_contingency(contingency)

print('\nConversion Rate Test (Guardrail Metric)')
print(f'Chi-Square Statistic: {chi2:.3f}, p-value: {p_val_conv:.4f}')

if p_val_conv < 0.05:
    print('Reject the null hypothesis: significant difference in conversion rate between groups.')
else:
    print('Fail to reject null hypothesis: no significant difference in conversion rate between groups.')

# -----------------------------
# 5. MDE AND DELTA SUMMARY
# -----------------------------
print(f'\nBaseline Revenue per user: €{base_rev_avg:.2f}')
print(f'MDE (30% of baseline): €{delta:.2f}')

# -----------------------------
# 6. POST-OBSERVED DATA
# -----------------------------
print("\nPost-Observed A/B Test Data (first 20 rows):")
print(df.head(20))


Required sample size per group: 463

Revenue per User Test (Success Metric)
T-Statistic: -1.154, p-value: 0.2513
Fail to reject null hypothesis: no significant increase in revenue per user.

Conversion Rate Test (Guardrail Metric)
Chi-Square Statistic: 0.161, p-value: 0.6886
Fail to reject null hypothesis: no significant difference in conversion rate between groups.

Baseline Revenue per user: €80.12
MDE (30% of baseline): €24.04

Post-Observed A/B Test Data (first 20 rows):
    user_id    group  base_price  items_bought  total_revenue  converted  \
0         1  Control   86.403163             1      86.403163          0   
1         2  Control   96.128513             1      96.128513          0   
2         3  Control  112.465079             2     224.930158          0   
3         4  Control   62.331645             2     124.663291          1   
4         5  Control   87.640680             2     175.281360          0   
5         6  Control  116.145651             1     116.145651   